# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [1]:
import pandas as pd
import csv
import re
import time
import multiprocessing as mp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/recipes_full.csv', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    abc = []
    for row in spamreader:
        abc.append([row[1], row[5], row[6]])

In [5]:
headers = abc.pop(0)
for i in abc:
    i[1] = i[1][1:-1]
    i[1] = re.sub(r"[''/""]+", "", str(i[1])).split(', ')

In [6]:
len(abc)/3
# будет 3 файла с одинаковым количеством строк

743879.0

In [7]:
abc1, abc2, abc3 = [], [], []
abc1 = abc[:743879]
abc2 = abc[743879:1487758]
abc3 = abc[1487758:2231637]

In [8]:
def f(listt, name):
    abe = []
    for i in listt:
        abe1 = []
        for j in i[1]:
            abe1.append([i[0], j, i[2]])
        abe.append(abe1)
    abee = []
    for i in abe:
        for j in i:
            abee.append(j)
    abee.insert(0, headers)
    with open(name, "w") as f:
        writer = csv.writer(f)
        writer.writerows(abee)
    return abee

In [ ]:
f(abc1, "id_tag_nsteps_1.csv")
f(abc2, "id_tag_nsteps_2.csv")
f(abc3, "id_tag_nsteps_3.csv")

In [9]:
p1 = mp.Process(target = f, args = (abc1, "id_tag_nsteps_1.csv"))
p1.start()
p2 = mp.Process(target = f, args = (abc2, "id_tag_nsteps_2.csv"))
p2.start()
p3 = mp.Process(target = f, args = (abc3, "id_tag_nsteps_3.csv"))
p3.start()
p1.join()
p2.join()
p3.join()

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [22]:
def f1(name, output):
    with open(name, encoding='utf-8') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        abc = []
        for row in spamreader:
            abc.append(row)
    abc = [x for x in abc if x]
    abcc = pd.DataFrame(abc)
    abcc.columns = abcc.iloc[0]
    abcc = abcc.drop(0)
    abcc.reset_index(inplace=True)
    abcc['n_steps'] =  abcc['n_steps'].astype('int')
    a = abcc.groupby(['tags']).mean()
    a.drop('index', axis=1, inplace=True)
    a = a.to_dict()
    output.put(a)

In [25]:
output = mp.Queue()
p1 = mp.Process(target = f1, args = ("id_tag_nsteps_1.csv", output,))
p1.start()
p2 = mp.Process(target = f1, args = ("id_tag_nsteps_2.csv", output,))
p2.start()
p3 = mp.Process(target = f1, args = ("id_tag_nsteps_3.csv", output,))
p3.start()
output.get()
output.get()
output.get()

{'n_steps': {'': 3.5195383347073372,
  '1-day-or-more': 4.529560685033507,
  '15-minutes-or-less': 4.976644721034168,
  '3-steps-or-less': 4.6989497695195555,
  '30-minutes-or-less': 7.609061435548077,
  '4-hours-or-less': 10.036623849948887,
  '5-ingredients-or-less': 5.335827053193245,
  '60-minutes-or-less': 9.376740544037041,
  'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.510538261997406,
  'a1-sauce': 3.516953316953317,
  'african': 4.359424460431654,
  'american': 7.624857468643102,
  'amish-mennonite': 3.5631146219571175,
  'angolan': 3.5005008347245408,
  'appetizers': 6.2362359993689855,
  'apples': 4.842016370014291,
  'april-fools-day': 3.509333333333333,
  'argentine': 3.576517812343201,
  'artichoke': 3.4956191105967926,
  'asian': 6.3985953434673855,
  'asparagus': 4.048851012022523,
  'australian': 4.22129735935706,
  'austrian': 3.6009876543209876,
  'avocado': 3.537479541734861,
  'bacon': 4.125410080524903,
  'baja': 3.5250573206681954,
 

In [ ]:
f1("id_tag_nsteps_2.csv")

{'n_steps': {'': 3.549801455989411,
  '1-day-or-more': 4.350454071758225,
  '15-minutes-or-less': 4.975564460952008,
  '3-steps-or-less': 4.73193742478941,
  '30-minutes-or-less': 7.607649071358749,
  '4-hours-or-less': 10.119333540124961,
  '5-ingredients-or-less': 5.334102679576672,
  '60-minutes-or-less': 9.449353375313155,
  'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.4950298210735586,
  'a1-sauce': 3.5310650887573964,
  'african': 4.361720554272518,
  'american': 7.523759565539373,
  'amish-mennonite': 3.545425048669695,
  'angolan': 3.4897327249022165,
  'appetizers': 6.274804381846636,
  'apples': 4.88111342351717,
  'april-fools-day': 3.497006983704689,
  'argentine': 3.587440250535685,
  'artichoke': 3.524112088628218,
  'asian': 6.494272300469484,
  'asparagus': 4.034838516746412,
  'australian': 4.244607420189819,
  'austrian': 3.568926735218509,
  'avocado': 3.5433526011560694,
  'bacon': 4.090338019742746,
  'baja': 3.562937062937063,
  'bake

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [ ]:
def f2(name1, name2, name3):
    with open(name1, encoding='utf-8') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        abc = []
        for row in spamreader:
            abc.append(row)
    with open(name2, encoding='utf-8') as csvfile1:
        spamreader1 = csv.reader(csvfile1, delimiter=',')
        abc1 = []
        for row in spamreader1:
            abc1.append(row)
    with open(name3, encoding='utf-8') as csvfile2:
        spamreader2 = csv.reader(csvfile2, delimiter=',')
        abc2 = []
        for row in spamreader2:
            abc2.append(row)
    abc = [x for x in abc if x]
    abc1 = [x for x in abc1 if x]
    abc2 = [x for x in abc2 if x]
    abcc = pd.DataFrame(abc)
    abcc.columns = abcc.iloc[0]
    abcc = abcc.drop(0)
    abcc.reset_index(inplace=True)
    abcc['n_steps'] =  abcc['n_steps'].astype('int')
    abcc1 = pd.DataFrame(abc1)
    abcc1.columns = abcc1.iloc[0]
    abcc1 = abcc1.drop(0)
    abcc1.reset_index(inplace=True)
    abcc1['n_steps'] =  abcc1['n_steps'].astype('int')
    abcc2 = pd.DataFrame(abc2)
    abcc2.columns = abcc2.iloc[0]
    abcc2 = abcc2.drop(0)
    abcc2.reset_index(inplace=True)
    abcc2['n_steps'] =  abcc2['n_steps'].astype('int')
    a = abcc.groupby(['tags']).mean()
    a.drop('index', axis=1, inplace=True)
    a = a.to_dict()
    a1 = abcc1.groupby(['tags']).mean()
    a1.drop('index', axis=1, inplace=True)
    a1 = a1.to_dict()
    a2 = abcc2.groupby(['tags']).mean()
    a2.drop('index', axis=1, inplace=True)
    a2 = a2.to_dict()
    abcc.append(abcc1)
    abcc.append(abcc2)
    a3 = abcc.groupby(['tags']).mean()
    a3.drop('index', axis=1, inplace=True)
    a3 = a3.to_dict()
    return a, a1, a2, a3

In [ ]:
start = time.process_time()
f2("id_tag_nsteps_1.csv", "id_tag_nsteps_2.csv", "id_tag_nsteps_3.csv")
new = (time.process_time() - start); new

54.65625

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 